In [152]:
import torch
import torch.nn as nn
# import pandas as pd
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import random
import re
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os

#from nltk.tokenize import word_tokenize
from collections import Counter

# from ...src.utils.wiki_tokens.wiki_cleaner import getMorphemeList
# import ../src/utiils/wiki_token/wiki_cleaner

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
with open("../tokens/wiki.json", "r") as file:
    tokenizer =json.load(file)

In [115]:
with open("text", "r") as file:
    wiki = file.read()

In [ ]:

# def getMorphemeList(text):
#   text = text.lower()
#   text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#   morpheme_list = word_tokenize(text)
  
#   # Count occurrences of each word
#   word_counts = Counter(morpheme_list)
  
#   # Filter out words that only appear once
#   morpheme_list = [word for word in morpheme_list if word_counts[word] > 1]
  
#   return morpheme_list


# words = getMorphemeList(wiki)

: 

In [116]:
words = wiki.split(sep=" ")
window = 2
print (words[0])
words.pop(0)

tokenized_wiki = []

for word in words:
    if word in tokenizer:
        tokenized_wiki.append(tokenizer[word])
        

In [121]:
print (tokenized_wiki[0:100])

[1246, 1958, 0, 32286, 21916, 149, 12200, 35001, 643, 10084, 36321, 5820, 32412, 21916, 32412, 27186, 1270, 32412, 27931, 7745, 21916, 32412, 27186, 36010, 32412, 32286, 17152, 30729, 35001, 15955, 0, 23223, 35831, 32826, 8642, 1553, 323, 32407, 35001, 35438, 32826, 8703, 32412, 22196, 21916, 29755, 17215, 999, 3012, 31898, 34899, 1958, 0, 24520, 17880, 4461, 28521, 8319, 32412, 36305, 1246, 17152, 8615, 12917, 32412, 36224, 27638, 5481, 17711, 1246, 1958, 0, 24294, 23666, 17152, 32412, 3066, 32407, 1813, 34562, 1270, 29042, 2924, 1018, 32479, 1813, 21916, 35971, 32568, 1246, 999, 32826, 26649, 29745, 32407, 530, 32412, 10411, 21916, 2329]


In [145]:
class embed_train_dataset(Dataset):
    def __init__(self, words):
        self.data = words
    
    def __len__(self):
        return len(self.data)-4
    
    def __getitem__(self, idx):
        idx = idx+2       
        sent = self.data[max(0,idx-window):min(idx+window+1,len(words))]    
        if len(sent) > 1:
            rand_idx = random.randint(0,len(sent)-1)
            target = sent[rand_idx]
            del sent[rand_idx]
            #print (sent)
            tokenized = torch.tensor(sent)

            
            return tokenized, torch.tensor(target)
        

In [135]:
dataset = embed_train_dataset(tokenized_wiki)
dataloader = DataLoader(dataset, batch_size=1,shuffle=True)

for data in dataloader:
    print (data)
    break



[17231, 12917, 22018, 17215]
[tensor([[17231, 12917, 22018, 17215]]), tensor([32412])]


In [136]:
class CBOW(torch.nn.Module):
    def __init__(self):
        super(CBOW, self).__init__()
        self.embeddings = torch.nn.Embedding(num_embeddings = len(tokenizer), embedding_dim = 200)
        self.linear = torch.nn.Linear(in_features = 200, out_features = len(tokenizer))
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = embeds.mean(dim=1)
        out = self.linear(embeds)
        probs =  F.log_softmax(out, dim=1)
        return probs

In [153]:
def train_loop():
    number_epochs = 5

    #train_wiki, val_wiki = train_test_split(words)
    os.makedirs("checkpoints", exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    dataset = embed_train_dataset(tokenized_wiki)
    dataloader = DataLoader(dataset, batch_size=128,shuffle=True)
    

    model = CBOW().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    best_loss = 100000000000000.0
    for epoch in range(number_epochs):
        model.train()
        epoch_loss = 0.0
        for X,Y in tqdm(dataloader):
            X = X.to(device)
            Y = Y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = F.cross_entropy(pred,Y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
        print(f"Epoch: {epoch}/{number_epochs}, loss: {epoch_loss} ")
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            save_path = f"checkpoints/storm_epoch_{epoch+1}.pt"
            torch.save(model.state_dict(), save_path)
            print(f"Model improved. Saved to: {save_path}")





In [154]:
train_loop()

  1%|          | 587/80027 [00:06<13:48, 95.87it/s] 


KeyboardInterrupt: 